# 1. Question 1 

Question 01: Sinh viên kiểm tra lại xem code đã đầy đủ các thành phần để hiểu dữ liệu này chưa? Nếu chưa thì bổ sung:
+ Kiểm tra khoảng thời gian dữ liệu phủ (start/end) và xác nhận tần suất theo giờ là liên tục.
+ Tính tỷ lệ thiếu theo từng biến, đồng thời quan sát thiếu theo thời gian (thiếu tập trung vào giai đoạn nào hay rải
đều).
+ Dùng boxplot hoặc quantile để nhìn nhanh ngoại lai (outliers) và phân phối lệch.
+ Vẽ chuỗi pm2.5 theo thời gian: một đồ thị toàn giai đoạn và một đồ thị phóng to 1–2 tháng để nhìn rõ dao động.
+ Kiểm tra tự tương quan đơn giản: so sánh tương quan của pm2.5 với các độ trễ như 24h và 168h để gợi ý chu kỳ
ngày/tuần.
+ Kiểm tra tính dừng: chạy adf/kpss và viết nhận xét ngắn gọn về kết luận.
Đồng thời, giải thích “thiếu theo biến nào là đáng lo nhất cho dự báo pm2.5 và vì sao?”

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adf, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import os

# Create images folder
if not os.path.exists('../images'):
    os.makedirs('../images')

# Colors
COLOR_BLUE = "#1F62FF"
COLOR_RED = "#FF351F" 
COLOR_GREEN = "#1FFF2A"
COLOR_ORANGE = "#FF9A1F"
COLOR_PURPLE = "#9E1FFF"
COLOR_YELLOW = "#FFDA1F"

# Font and Style Settings
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12

# 1. Load Data
# Assuming we are running from 'notebooks/' directory, so path to data is '../data/'
try:
    df = pd.read_csv('../data/processed/01_cleaned.csv')
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)
    print("Data loaded. Shape:", df.shape)
    print("Date Range:", df.index.min(), "to", df.index.max())
    
    # Check continuity
    full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
    missing_timestamps = full_idx.difference(df.index)
    print(f"Missing Hours count: {len(missing_timestamps)}")
    if len(missing_timestamps) == 0:
        print("Conclusion: Data is continuous hourly.")
    else:
        print("Conclusion: Data has gaps.")
        
except Exception as e:
    print(f"Error loading data: {e}")

# 2. Missing Analysis
# Since input is 'cleaned', it might be 0 missing. But checking anyway.
missing_rates = df.isna().mean() * 100
print("\nMissing Rates (%):\n", missing_rates[missing_rates > 0])

# 3. Outliers (Boxplot)
plt.figure(figsize=(10, 6))
sns.boxplot(y=df['PM2.5'], color=COLOR_BLUE)
plt.title('Distribution of PM2.5 (Boxplot)')
plt.ylabel('PM2.5 (ug/m3)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout(pad=1)
plt.savefig('../images/01_outlier_boxplot.png')
plt.show()

# 4. Full Time Series
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['PM2.5'], color=COLOR_BLUE, linewidth=0.8, label='PM2.5')
plt.title('PM2.5 Concentration - Full Period (2013-2017)')
plt.xlabel('Date (Ox)')
plt.ylabel('PM2.5 (ug/m3) (Oy)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout(pad=1)
plt.savefig('../images/01_pm25_full.png')
plt.show()

# 5. Zoom Time Series (Jan-Feb 2016)
zoom_start, zoom_end = '2016-01-01', '2016-02-29'
df_zoom = df[zoom_start:zoom_end]
plt.figure(figsize=(15, 6))
plt.plot(df_zoom.index, df_zoom['PM2.5'], color=COLOR_ORANGE, linewidth=1.5, label='PM2.5 Zoom')
plt.title(f'PM2.5 Concentration Zoom ({zoom_start} to {zoom_end})')
plt.xlabel('Date (Ox)')
plt.ylabel('PM2.5 (ug/m3) (Oy)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout(pad=1)
plt.savefig('../images/01_pm25_zoom.png')
plt.show()

# 6. Autocorrelation (Lag 24h & 168h)
lag24 = df['PM2.5'].autocorr(lag=24)
lag168 = df['PM2.5'].autocorr(lag=168)
print(f"\nAutocorrelation Lag 24h: {lag24:.4f}")
print(f"Autocorrelation Lag 168h: {lag168:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
plot_acf(df['PM2.5'], lags=50, ax=axes[0], color=COLOR_BLUE, title='Autocorrelation (ACF)')
plot_pacf(df['PM2.5'], lags=50, ax=axes[1], color=COLOR_RED, title='Partial Autocorrelation (PACF)')
plt.tight_layout(pad=1)
plt.savefig('../images/01_acf_pacf.png')
plt.show()

# 7. Stationarity Tests
print("\nRunning ADF Test...")
adf_res = adf(df['PM2.5'].dropna())
print(f"ADF Statistic: {adf_res[0]:.4f}, p-value: {adf_res[1]:.4f}")
if adf_res[1] < 0.05:
    print("Result: Stationary (Reject H0)")
else:
    print("Result: Non-Stationary (Fail to Reject H0)")

print("\nExplanation: PM2.5 missing is most worrying because it breaks the autoregressive chain.")


# 2. Question 2

Question 02: Đối với baseline hồi quy cho dự báo, sinh viên cần giải thích được:
+ Vì sao lag 24h thường có ý nghĩa: do nhịp sinh hoạt theo ngày và các điều kiện khí tượng lặp lại theo chu kỳ ngày.
+ Vì sao phải chia theo thời gian bằng cutoff: để mô hình không “nhìn thấy tương lai”, tránh đánh giá đẹp giả do
leakage.
+ Phân biệt rmse và mae: khi nào rmse cao hơn nhiều? thường xảy ra khi dữ liệu có spike hoặc dự báo sai mạnh ở
một số thời điểm, vì rmse phạt nặng sai số lớn

In [ ]:
# Code for Question 2 Illustration

# 1. Lag 24 visual check
df['PM2.5_lag24'] = df['PM2.5'].shift(24)
plt.figure(figsize=(8, 8))
plt.scatter(df['PM2.5_lag24'], df['PM2.5'], alpha=0.2, color=COLOR_PURPLE, s=10)
plt.title('Correlation: PM2.5(t) vs PM2.5(t-24h)')
plt.xlabel('PM2.5 at t-24h (Ox)')
plt.ylabel('PM2.5 at t (Oy)')
plt.grid(True, alpha=0.3)
plt.tight_layout(pad=1)
plt.savefig('../images/02_lag24_scatter.png')
plt.show()

corr_24 = df['PM2.5'].corr(df['PM2.5_lag24'])
print(f"Correlation Lag 24: {corr_24:.4f}")

# Explanation is in Markdown above, but we confirm logic here.
print("\nRMSE vs MAE logic:")
print("RMSE squares errors -> penalizes outliers/spikes heavily.")
print("MAE averages absolute errors -> more robust to spikes.")


# 3. Question 3

Question 03: Đối với model dự báo bằng ARIMA, sinh viên cần giải thích được: quy trình ra quyết định arima mà sinh
viên phải viết trong báo cáo
+ Quan sát chuỗi gốc để nhận diện xu hướng và mùa vụ (nếu có).
+ Kiểm định dừng bằng adf/kpss để chọn 𝑑.
+ Xem acf/pacf để đoán các giá trị ứng viên cho 𝑝 và 𝑞.
+ grid search với 𝑝, 𝑞 nhỏ (giữ 𝑑 cố định) và chọn mô hình theo aic/bic.
+ Chẩn đoán phần dư: kiểm tra residual có gần “white noise” hay không, nhằm đánh giá mô hình đã bắt được cấu trúc chính 
của chuỗi chưa.

In [ ]:
# Code for Question 3 Illustration: ARIMA Steps
print("ARIMA Decision Process Demonstration:")

# Step 1: Trend & Seasonality
print("1. Trend/Seasonality: Seen in Question 1. Seasonality is present (24h).")

# Step 2: Stationarity (d)
print(f"2. Stationarity (d): ADF p-value={adf_res[1]:.4f}. If < 0.05 -> d=0.")

# Step 3: p and q candidates
print("3. p, q selection: Refer to PACF/ACF plots in Q1.")
print("   PACF Spikes at 1, 2 -> Suggest p=1 or p=2")
print("   ACF Decays -> Suggest AR process")

# Step 4: Grid Search (Conceptual)
print("4. Grid Search: Try p in [0..3], q in [0..3], d=0. Metric: AIC.")

# Step 5: Residuals
print("5. Residuals: Verify they are uncorrelated (White Noise).")
